# Configuration et Installation TweetyProject

**Navigation**: Index | [Suivant: Tweety-2-Basic-Logics](Tweety-2-Basic-Logics.ipynb)

---

Ce notebook configure l'environnement pour la série de notebooks TweetyProject qui explore la bibliothèque Java [TweetyProject](https://tweetyproject.org/) pour l'intelligence artificielle symbolique.

**IMPORTANT:** Ce notebook **Tweety-1-Setup** configure uniquement l'environnement. Les exemples de logiques et d'argumentation se trouvent dans les notebooks suivants de la série.

## Objectifs pédagogiques

1. Comprendre l'architecture TweetyProject (JARs Java + JPype)
2. Configurer l'environnement Python/Java pour Tweety
3. Vérifier le bon fonctionnement de la JVM et des imports

## Versions TweetyProject

| Version | Date | Nouveautés | Notes |
|---------|------|------------|-------|
| **1.28** | Janvier 2025 | arg.caf, k-admissibility | Version stable |
| **1.29** | Juillet 2025 | arg.eaf (Epistemic AF), graph rendering | Recommandée |

> **Configuration**: La version peut être changée dans la cellule de téléchargement des JARs (section 1.4).

## Notes techniques (Tweety 1.29)

| Fonctionnalité | Statut | Notes |
|----------------|--------|-------|
| **CrMas/InformationObject** | OK | Package `org.tweetyproject.beliefdynamics.mas` |
| **ADF natif SAT** | OK | NativeMinisatSolver via `libs/native/` |
| **SPASS** | OK | Requiert droits admin sur Windows |
| **SimpleMlReasoner** | Limité | Préférer SPASS externe |
| **AF Learning** | Désactivé | Bug interne ClassCastException |

## Série complète Tweety (10 notebooks)

| # | Notebook | Thème | Prérequis |
|---|----------|-------|-----------|
| 1 | **Ce notebook** | Configuration JVM/JPype | - |
| 2 | [Tweety-2-Basic-Logics](Tweety-2-Basic-Logics.ipynb) | Logique Propositionnelle, FOL | Setup |
| 3 | [Tweety-3-Advanced-Logics](Tweety-3-Advanced-Logics.ipynb) | DL, Modale, QBF, Conditional | Setup |
| 4 | [Tweety-4-Belief-Revision](Tweety-4-Belief-Revision.ipynb) | CrMas, MUS, MaxSAT | Setup |
| 5 | [Tweety-5-Abstract-Argumentation](Tweety-5-Abstract-Argumentation.ipynb) | Dung AF, CF2, Génération | Setup |
| 6 | [Tweety-6-Structured-Argumentation](Tweety-6-Structured-Argumentation.ipynb) | ASPIC+, DeLP, ABA, ASP | Setup + Clingo |
| 7a | [Tweety-7a-Extended-Frameworks](Tweety-7a-Extended-Frameworks.ipynb) | ADF, Bipolar, WAF, SAF, SetAF | Setup |
| 7b | [Tweety-7b-Ranking-Probabilistic](Tweety-7b-Ranking-Probabilistic.ipynb) | Ranking Semantics, Probabiliste | Setup |
| 8 | [Tweety-8-Agent-Dialogues](Tweety-8-Agent-Dialogues.ipynb) | Agents, Dialogues argumentatifs | Setup |
| 9 | [Tweety-9-Preferences](Tweety-9-Preferences.ipynb) | Préférences, Théorie du vote | Setup |

---

## Plan de ce Notebook

**Section 1 : [Installation et Configuration](#partie1)**
* 1.2 [Pré-requis](#1.2)
* 1.3 [Installation des Packages Python](#1.3)
* 1.4 [Téléchargement des JARs Tweety](#1.4)
* 1.5 [Configuration des Outils Externes](#1.5)
* 1.6 [Démarrage de la JVM](#1.6)
* 1.7 [Concepts Clés](#1.7)

---

## Partie 1 : Introduction et Configuration
<a id="partie1"></a>

Cette section couvre la mise en place de l'environnement nécessaire pour exécuter les exemples de ce notebook.

### 1.2 Pré-requis
<a id="1.2"></a>

*   **Python 3.x** : Assurez-vous d'avoir une installation Python fonctionnelle (testé avec 3.10+).
*   **Java Development Kit (JDK)** : JPype nécessite un JDK (version 11 ou supérieure recommandée) pour démarrer la JVM.
    *   Vérifiez votre installation avec `java -version` dans un terminal.
    *   Assurez-vous que la variable d'environnement `JAVA_HOME` pointe vers le répertoire racine de votre installation JDK (par exemple, `/usr/lib/jvm/java-17-openjdk-amd64` sur Linux, `C:\Program Files\Java\jdk-17` sur Windows).
    *   Si `JAVA_HOME` n'est pas définie, le script de démarrage de la JVM (section 1.6) essaiera de trouver un JDK automatiquement, mais il est préférable de la définir explicitement.
*   **(Optionnel) Outils Externes** : Pour certaines fonctionnalités avancées (voir section [1.5](#1.5)), vous devrez installer des outils spécifiques et configurer leurs chemins d'accès plus loin dans le notebook.
*   **(Optionnel) Graphviz**: Pour visualiser certains graphes d'argumentation (non implémenté dans ce tutoriel mais possible).

### 1.3 Installation des Packages Python
<a id="1.3"></a>

Nous utilisons `jpype1` pour le pont Java-Python. Les autres packages (`requests`, `tqdm`) sont des utilitaires pour le téléchargement des JARs.

In [1]:
# Vérification/Installation des packages Python nécessaires
import importlib
import sys
import subprocess

# Le package s'appelle 'jpype1' sur PyPI, mais on importe 'jpype'
# z3 s'importe comme 'z3' mais s'installe comme 'z3-solver'
# pysat s'importe comme 'pysat' mais s'installe comme 'python-sat'
packages_to_check = {
    'jpype': 'jpype1',
    'requests': 'requests',
    'tqdm': 'tqdm',
    'clingo': 'clingo',
    'z3': 'z3-solver',       # Pour MARCO (MUS enumeration) et SMT solving
    'pysat': 'python-sat'    # Pour MaxSAT (RC2) et solveurs SAT modernes (CaDiCaL)
}
packages_to_install = []
all_found = True

print("--- Vérification des packages Python requis ---")
for import_name, install_name in packages_to_check.items():
    try:
        importlib.import_module(import_name)
        print(f"✔️ {import_name} trouvé.")
    except ImportError:
        print(f"⚠️ {import_name} manquant (package: {install_name}).")
        packages_to_install.append(install_name)
        all_found = False

if packages_to_install:
    print(f"\nTentative d'installation des packages manquants: {', '.join(packages_to_install)}...")
    try:
        # Utiliser -q pour une sortie moins verbeuse, retirer si le debug est nécessaire
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + packages_to_install)
        print(f"✅ Packages {', '.join(packages_to_install)} installés.")
        print("\n‼️ IMPORTANT : Vous devez probablement REDÉMARRER LE NOYAU (Kernel -> Restart Kernel) pour que les nouveaux packages soient pris en compte.")
        # Marquer comme non trouvés pour l'instant, car le noyau doit redémarrer
        all_found = False
    except Exception as e:
        print(f"❌ Échec de l'installation automatique : {e}")
        print(f"   Veuillez installer manuellement : pip install {' '.join(packages_to_install)}")
        all_found = False # Échec -> pas trouvé

if all_found:
    print("\n✔️ Tous les packages Python requis sont présents.")

# Importations pour vérifier après redémarrage potentiel (ne lèvera pas d'erreur ici)
try:
    import jpype
    import requests
    import tqdm
except ImportError:
    if not packages_to_install: # Si on n'a pas essayé d'installer, c'est une autre erreur
         print("\n⚠️ Erreur d'import inattendue. Vérifiez votre environnement Python.")
    # Si on a installé, le message d'erreur est déjà affiché.
    pass

--- Vérification des packages Python requis ---
✔️ jpype trouvé.
✔️ requests trouvé.
✔️ tqdm trouvé.
✔️ clingo trouvé.
✔️ z3 trouvé.
✔️ pysat trouvé.

✔️ Tous les packages Python requis sont présents.


### 1.4 Téléchargement des JARs Tweety et Dépendances
<a id="1.4"></a>

TweetyProject est une collection de bibliothèques Java distribuées sous forme de fichiers JAR. Pour utiliser Tweety avec JPype, nous devons télécharger ces JARs et les rendre accessibles à la JVM.

Le script suivant va :
1.  Créer un sous-dossier `libs/` s'il n'existe pas.
2.  Vérifier si l'URL de base pour la version spécifiée de Tweety (`TWEETY_VERSION`) est accessible.
3.  Télécharger (ou vérifier la présence) du JAR **principal** `tweety-full-...-with-dependencies.jar`. Celui-ci contient le noyau de Tweety et de nombreuses dépendances courantes.
4.  Télécharger (ou vérifier la présence) des JARs **spécifiques aux modules** utilisés dans ce notebook (argumentation, logiques spécifiques, etc.). La liste `REQUIRED_MODULES` définit les modules nécessaires.

*Note : Le téléchargement peut prendre quelques minutes lors de la première exécution. Assurez-vous que tous les JARs nécessaires sont bien présents dans le dossier `libs/` avant de démarrer la JVM, car des JARs manquants causeront des erreurs d'import plus loin.*

In [2]:
import pathlib
import urllib.request
import os
import requests
from tqdm.auto import tqdm # Barre de progression plus jolie dans les notebooks

# --- Configuration TweetyProject ---
# Versions disponibles:
#   - 1.28 (Janvier 2025): Version stable avec arg.caf, k-admissibility
#   - 1.29 (Juillet 2025): Ajoute arg.eaf (Epistemic AF), graph rendering, bugfixes mineurs
#
# Limitations connues (présentes dans 1.28, non confirmées corrigées en 1.29):
#   - CrMas/InformationObject: API refactorisée, certaines classes manquantes
#   - AF Learning: ClassCastException interne
#   - SimpleMlReasoner: Bloque sans SPASS externe
#   - ADF: Nécessite solveur SAT natif (Minisat) non disponible via JPype
#
# Recommandation: Utiliser 1.28 pour la stabilité, 1.29 pour arg.eaf ou les bugfixes
TWEETY_VERSION = "1.29"  # Changer en "1.29" pour utiliser la dernière version

BASE_URL = f"https://tweetyproject.org/builds/{TWEETY_VERSION}/"
LIB_DIR = pathlib.Path("libs")
LIB_DIR.mkdir(exist_ok=True)
CORE_JAR_NAME = f"org.tweetyproject.tweety-full-{TWEETY_VERSION}-with-dependencies.jar"
CORE_JAR_PATH = LIB_DIR / CORE_JAR_NAME

# Modules Tweety requis (couvre tous les exemples de la série)
# Note: arg.eaf (Epistemic AF) est uniquement disponible en 1.29+
REQUIRED_MODULES_BASE = sorted([
    "arg.adf", "arg.aba", "arg.bipolar", "arg.aspic", "arg.dung", "arg.weighted",
    "arg.social", "arg.setaf", "arg.rankings", "arg.prob", "arg.extended",
    "arg.delp", "arg.deductive", "arg.caf",
    "beliefdynamics", "agents.dialogues", "action", "preferences",
    "logics.pl", "logics.fol", "logics.ml", "logics.dl", "logics.cl",
    "logics.qbf", "logics.pcl", "logics.rcl", "logics.rpcl", "logics.mln", "logics.bpm",
    "lp.asp",
    "math", "commons", "agents"
])

# Ajouter arg.eaf pour version 1.29+
REQUIRED_MODULES = REQUIRED_MODULES_BASE.copy()
if TWEETY_VERSION >= "1.29":
    REQUIRED_MODULES.append("arg.eaf")
    REQUIRED_MODULES = sorted(REQUIRED_MODULES)
    print(f"Version {TWEETY_VERSION}: Module arg.eaf (Epistemic AF) inclus.")

# Vérification accessibilité URL de base
print(f"Vérification de l'accès à {BASE_URL}...")
url_accessible = False
try:
    response = requests.head(BASE_URL, timeout=10)
    response.raise_for_status()
    print(f"✔️ URL de base Tweety v{TWEETY_VERSION} accessible.")
    url_accessible = True
except requests.exceptions.RequestException as e:
    print(f"❌ Impossible d'accéder à l'URL de base {BASE_URL}. Vérifiez la version ou votre connexion. Erreur : {e}")
    print("   Le téléchargement des JARs manquants échouera. Seuls les JARs locaux seront utilisables.")

# Barre de progression TQDM
class TqdmUpTo(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

# Fonction de téléchargement
def download_jar(jar_name, url_base, target_dir):
    jar_path = target_dir / jar_name
    url = url_base + jar_name
    newly_downloaded = False

    if not jar_path.exists():
        if not url_accessible:
            print(f"   ⏭️ '{jar_name}' manquant, mais URL inaccessible. Téléchargement sauté.")
            return False, False
        try:
            response = requests.head(url, timeout=5)
            response.raise_for_status()
            with TqdmUpTo(unit='B', unit_scale=True, unit_divisor=1024, miniters=1, desc=jar_name[:40]) as t:
                urllib.request.urlretrieve(url, filename=jar_path, reporthook=t.update_to)
            if jar_path.exists() and jar_path.stat().st_size > 0:
                newly_downloaded = True
            else:
                print(f"\n❓ Téléchargement de {jar_name} terminé mais fichier vide ou absent.")
                if jar_path.exists(): jar_path.unlink(missing_ok=True)
        except requests.exceptions.HTTPError as e:
            print(f"\n❌ {jar_name} non trouvé sur le serveur (Erreur {e.response.status_code}).")
            return False, False
        except Exception as e:
            print(f"\n❌ Erreur lors du téléchargement de {jar_name}: {e}")
            if jar_path.exists(): jar_path.unlink(missing_ok=True)
            return False, False

    return newly_downloaded, jar_path.exists()

# --- Exécution du Téléchargement ---

# 1. JAR Core
print(f"\n--- Vérification/Téléchargement JAR Core ---")
core_new, core_exists = download_jar(CORE_JAR_NAME, BASE_URL, LIB_DIR)
if core_exists:
    status = "téléchargé" if core_new else "déjà présent"
    print(f"✔️ JAR Core '{CORE_JAR_NAME}' {status}.")
else:
     print(f"❌ ERREUR CRITIQUE : Le JAR core {CORE_JAR_NAME} est manquant et n'a pas pu être téléchargé.")

# 2. JARs Modules
print(f"\n--- Vérification/Téléchargement des {len(REQUIRED_MODULES)} JARs de modules ---")
modules_downloaded_count = 0
modules_present_count = 0
modules_missing = []

if core_exists:
    with tqdm(REQUIRED_MODULES, desc="Modules") as pbar:
        for module in pbar:
            pbar.set_postfix_str(module, refresh=True)
            module_jar_name = f"org.tweetyproject.{module}-{TWEETY_VERSION}-with-dependencies.jar"
            new_dl, present = download_jar(module_jar_name, BASE_URL, LIB_DIR)
            if new_dl:
                modules_downloaded_count += 1
            if present:
                modules_present_count += 1
            else:
                modules_missing.append(module_jar_name)
else:
    print("\nSkipping module download because Core JAR is missing.")
    modules_present_count = 0
    for module in REQUIRED_MODULES:
         module_jar_name = f"org.tweetyproject.{module}-{TWEETY_VERSION}-with-dependencies.jar"
         if (LIB_DIR / module_jar_name).exists():
              modules_present_count += 1
         else:
              modules_missing.append(module_jar_name)


print(f"\n--- Résumé Téléchargement ---")
print(f"  Version Tweety: {TWEETY_VERSION}")
print(f"  JAR Core: {'Présent' if core_exists else 'MANQUANT'}")
print(f"  JARs Modules:")
print(f"    - Nouveaux téléchargés : {modules_downloaded_count}")
print(f"    - Total présents       : {modules_present_count} / {len(REQUIRED_MODULES)}")
if modules_missing:
    print(f"    - ⚠️ Modules MANQUANTS : {', '.join(modules_missing)}")
print(f"  Chemin du dossier libs : {LIB_DIR.resolve()}")

# Vérification finale
all_jars = list(LIB_DIR.glob("*.jar"))
if not core_exists or modules_present_count < len(REQUIRED_MODULES):
    print("\n⚠️ Des JARs Tweety semblent manquants. Vérifiez les messages d'erreur ci-dessus.")
    print("   Le démarrage de la JVM ou l'utilisation de certaines fonctionnalités échoueront probablement.")
else:
    print("\n✔️ Vérification des JARs Tweety terminée. Le démarrage de la JVM peut continuer.")

Version 1.29: Module arg.eaf (Epistemic AF) inclus.
Vérification de l'accès à https://tweetyproject.org/builds/1.29/...
✔️ URL de base Tweety v1.29 accessible.

--- Vérification/Téléchargement JAR Core ---
✔️ JAR Core 'org.tweetyproject.tweety-full-1.29-with-dependencies.jar' déjà présent.

--- Vérification/Téléchargement des 34 JARs de modules ---


Modules:   0%|          | 0/34 [00:00<?, ?it/s]


--- Résumé Téléchargement ---
  Version Tweety: 1.29
  JAR Core: Présent
  JARs Modules:
    - Nouveaux téléchargés : 0
    - Total présents       : 34 / 34
  Chemin du dossier libs : D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\Tweety\libs

✔️ Vérification des JARs Tweety terminée. Le démarrage de la JVM peut continuer.


#### Fichiers de Données Complémentaires

En plus des JARs, TweetyProject utilise des fichiers de données d'exemple pour les différentes logiques et systèmes d'argumentation :

- **DeLP** : `birds.txt`, `nixon.txt` (exemples de raisonnement défaisable)
- **ABA** : `example1.aba`, `example2.aba` (Assumption-Based Argumentation)
- **ASPIC+** : `ex1.aspic` (règles strictes et défaisables)
- **Logiques** : fichiers `.proplogic`, `.fologic`, `.mlogic`, `.qbf`

Ces fichiers sont téléchargés depuis le dépôt GitHub officiel de TweetyProject.

In [3]:
# --- 1.4bis Téléchargement des Fichiers de Données (.txt, .aba, etc.) ---
print("\n--- 1.4bis Téléchargement des Fichiers de Données ---")

import pathlib
import requests
import os
from tqdm.auto import tqdm

# Créer le dossier de destination s'il n'existe pas
RESOURCE_DIR = pathlib.Path("resources")
RESOURCE_DIR.mkdir(exist_ok=True)
print(f"ℹ️ Les fichiers de données seront téléchargés dans: {RESOURCE_DIR.resolve()}")

# Base URL pour les fichiers bruts sur GitHub (branche main)
GITHUB_RAW_BASE_URL = "https://raw.githubusercontent.com/TweetyProjectTeam/TweetyProject/main/"

# Mapper les noms de fichiers aux chemins relatifs probables dans le dépôt GitHub
# (Liste inchangée par rapport à votre version)
FILES_TO_DOWNLOAD = {
    # DeLP Files
    "birds.txt": "org-tweetyproject-arg-delp/src/main/resources/birds.txt",
    "birds2.txt": "org-tweetyproject-arg-delp/src/main/resources/birds2.txt",
    "nixon.txt": "org-tweetyproject-arg-delp/src/main/resources/nixon.txt",
    "counterarg.txt": "org-tweetyproject-arg-delp/src/main/resources/counterarg.txt",
    # ABA Files
    "example1.aba": "org-tweetyproject-arg-aba/src/main/resources/example1.aba",
    "example2.aba": "org-tweetyproject-arg-aba/src/main/resources/example2.aba",
    "example5.aba": "org-tweetyproject-arg-aba/src/main/resources/example5.aba",
    "smp_fol.aba": "org-tweetyproject-arg-aba/src/main/resources/smp_fol.aba",
    # ASPIC Files
    "ex1.aspic": "org-tweetyproject-arg-aspic/src/main/resources/ex1.aspic",
    "ex5_fol.aspic": "org-tweetyproject-arg-aspic/src/main/resources/ex5_fol.aspic",
    # Autres Logiques/Args
     "examplebeliefbase.proplogic": "org-tweetyproject-logics-pl/src/main/resources/examplebeliefbase.proplogic",
     "examplebeliefbase_multiple.proplogic": "org-tweetyproject-logics-pl/src/main/resources/examplebeliefbase_multiple.proplogic",
     "examplebeliefbase_xor.proplogic": "org-tweetyproject-logics-pl/src/main/resources/examplebeliefbase_xor.proplogic",
     "dimacs_ex4.cnf": "org-tweetyproject-logics-pl/src/main/resources/dimacs_ex4.cnf",
     "examplebeliefbase.dlogic": "org-tweetyproject-logics-dl/src/main/resources/examplebeliefbase.dlogic",
     "examplebeliefbase2.fologic": "org-tweetyproject-logics-fol/src/main/resources/examplebeliefbase2.fologic",
     "examplebeliefbase.mlogic": "org-tweetyproject-logics-ml/src/main/resources/examplebeliefbase.mlogic",
     "examplebeliefbase2.mlogic": "org-tweetyproject-logics-ml/src/main/resources/examplebeliefbase2.mlogic",
     "tweety-example.qbf": "org-tweetyproject-logics-qbf/src/main/resources/tweety-example.qbf",
     "qdimacs-example1.qdimacs": "org-tweetyproject-logics-qbf/src/main/resources/qdimacs-example1.qdimacs",
     "qcir-example1.qcir": "org-tweetyproject-logics-qbf/src/main/resources/qcir-example1.qcir",
     "qcir-example2-sat.qcir": "org-tweetyproject-logics-qbf/src/main/resources/qcir-example2-sat.qcir",
     # BPMN - Gardons celui de votre TP comme exemple
     "problematic_hit.bpmn": "org-tweetyproject-logics-bpm/src/main/resources/problematic_hit.bpmn",
     "monkey.desc": "org-tweetyproject-action/src/main/resources/monkey.desc",
     "conditioner.desc": "org-tweetyproject-action/src/main/resources/conditioner.desc",
     "adf_example.txt": "org-tweetyproject-arg-adf/src/main/resources/adf_example.txt",
     "ex1.apx": "org-tweetyproject-arg-setaf/src/main/resources/ex1.apx",
     "ex1_bonzon.apx": "org-tweetyproject-arg-rankings/src/main/resources/ex1_bonzon.apx",
     "ex1.tgf": "org-tweetyproject-arg-dung/src/main/resources/ex1.tgf",
}

# Barre de progression TQDM (inchangée)
class TqdmUpTo(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None: self.total = tsize
        self.update(b * bsize - self.n)

# Boucle de téléchargement
downloaded_count = 0
present_count = 0
failed_count = 0
skipped_count = 0

print(f"\nVérification/Téléchargement de {len(FILES_TO_DOWNLOAD)} fichiers de données...")

# Vérification de l'accessibilité GitHub avec un fichier test (README.md)
github_accessible = False
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
try:
    # Tester avec README.md car la racine n'est pas accessible sur raw.githubusercontent.com
    test_url = GITHUB_RAW_BASE_URL + "README.md"
    response_gh = requests.head(test_url, timeout=10, headers=headers, allow_redirects=True)
    github_accessible = response_gh.ok
    if not github_accessible:
        print(f"⚠️ Impossible d'accéder a GitHub {test_url}. Code: {response_gh.status_code}")
    else:
        print("✔️ Accès à la base GitHub Raw confirmé.")
except requests.exceptions.RequestException as e_gh:
    print(f"⚠️ Erreur de connexion à GitHub: {e_gh}")

if not github_accessible:
    print("   Skipping all file downloads from GitHub.")
    skipped_count = len(FILES_TO_DOWNLOAD)
    # Compter ceux déjà présents localement
    for filename in FILES_TO_DOWNLOAD:
        if (RESOURCE_DIR / filename).exists():
             present_count += 1
else:
    with tqdm(FILES_TO_DOWNLOAD.items(), desc="Fichiers Données") as pbar:
        for filename, relative_path in pbar:
            pbar.set_postfix_str(filename, refresh=True)
            target_path = RESOURCE_DIR / filename
            file_url = GITHUB_RAW_BASE_URL + relative_path

            if target_path.exists() and target_path.stat().st_size > 0:
                present_count += 1
                continue # Skip download if already present and not empty

            try:
                # Utiliser GET avec header pour le téléchargement aussi
                response = requests.get(file_url, stream=True, timeout=15, headers=headers) # Timeout augmenté
                # Vérifier explicitement 404
                if response.status_code == 404:
                    print(f"\n   ⚠️ Fichier '{filename}' non trouvé sur GitHub (404) à l'URL: {file_url}")
                    failed_count += 1
                    continue
                response.raise_for_status() # Lève une exception pour les autres erreurs HTTP

                total_size = int(response.headers.get('content-length', 0))
                with TqdmUpTo(unit='B', unit_scale=True, unit_divisor=1024, total=total_size, desc=filename, leave=False) as t:
                    with open(target_path, 'wb') as f:
                        for chunk in response.iter_content(chunk_size=8192):
                            if chunk:
                                f.write(chunk)
                                t.update(len(chunk))

                if target_path.exists() and target_path.stat().st_size > 0:
                    downloaded_count += 1
                    present_count += 1
                else:
                    print(f"\n   ❓ Téléchargement de '{filename}' terminé mais fichier vide/absent.")
                    if target_path.exists(): target_path.unlink(missing_ok=True)
                    failed_count += 1

            except requests.exceptions.RequestException as e:
                print(f"\n   ❌ Échec téléchargement '{filename}' : {e}")
                if target_path.exists(): target_path.unlink(missing_ok=True)
                failed_count += 1
            except Exception as e_other:
                 print(f"\n   ❌ Erreur inattendue pour '{filename}': {e_other}")
                 if target_path.exists(): target_path.unlink(missing_ok=True)
                 failed_count += 1


# Résumé
print(f"\n--- Résumé Téléchargement Données ---")
print(f"   Dossier cible          : {RESOURCE_DIR.resolve()}")
print(f"   Fichiers téléchargés  : {downloaded_count}")
print(f"   Fichiers déjà présents : {present_count - downloaded_count}")
if skipped_count > 0: print(f"   Fichiers sautés (connexion échouée): {skipped_count}")
print(f"   Total fichiers OK      : {present_count} / {len(FILES_TO_DOWNLOAD)}")
if failed_count > 0:
     print(f"   ⚠️ Échecs (Non trouvés sur GitHub ou autre erreur): {failed_count}")
     print("      Les sections utilisant ces fichiers pourraient échouer.")
elif skipped_count == 0 and present_count == len(FILES_TO_DOWNLOAD):
     print("✔️ Tous les fichiers de données nécessaires semblent présents.")


--- 1.4bis Téléchargement des Fichiers de Données ---
ℹ️ Les fichiers de données seront téléchargés dans: D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\Tweety\resources

Vérification/Téléchargement de 29 fichiers de données...
✔️ Accès à la base GitHub Raw confirmé.


Fichiers Données:   0%|          | 0/29 [00:00<?, ?it/s]


--- Résumé Téléchargement Données ---
   Dossier cible          : D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\Tweety\resources
   Fichiers téléchargés  : 0
   Fichiers déjà présents : 29
   Total fichiers OK      : 29 / 29
✔️ Tous les fichiers de données nécessaires semblent présents.


### 1.5 Configuration des Outils Externes (Optionnel)
<a id="1.5"></a>

Certaines fonctionnalités avancées de Tweety s'appuient sur des outils externes (solveurs SAT/MaxSAT, prouveurs FOL/ML, énumérateurs MUS, solveurs ASP). Pour utiliser ces fonctionnalités, vous devez :

1.  **Installer l'outil externe** séparément en suivant les instructions de son site web.
2.  **Indiquer le chemin d'accès** à l'exécutable de l'outil dans la cellule de code ci-dessous.

**Outils potentiellement utilisés dans ce notebook :**

*   **Solveurs SAT externes** (pour `logics.pl.sat.CmdLineSatSolver` - Section 2.1.3) :
    *   Lingeling, CaDiCaL, Kissat, MiniSat, Glucose, etc. (prennent souvent le format DIMACS)
*   **Prouveur FOL** (pour `logics.fol.reasoner.EFOLReasoner` - Section 2.2.2) :
    *   **EProver** : Téléchargement et compilation peuvent être complexes. ([Site Eprover](https://eprover.org/))
    *   *Alternative* : Tweety peut aussi utiliser SPASS pour certains fragments FOL, configuré ci-dessous.
*   **Prouveur ML** (pour `logics.ml.reasoner.SPASSMlReasoner` - Section 2.4.2) :
    *   **SPASS** : ([Site SPASS](https://www.spass-prover.org/)) - Binaires souvent disponibles.
*   **Énumérateur MUS** (pour `logics.pl.sat.MarcoMusEnumerator` - Section 3.3) :
    *   **MARCO** : Script Python. ([Lien potentiel via recherche](https://github.com/marcomus/marco))
*   **Solveur MaxSAT** (pour `logics.pl.sat.OpenWboSolver` - Section 3.4) :
    *   **Open-WBO** : ([Site Open-WBO](https://github.com/sat-group/open-wbo))
*   **Solveur ASP** (pour `lp.asp.reasoner.ClingoSolver` - Section 4.6) :
    *   **Clingo** (partie de Potassco) : ([Site Potassco](https://potassco.org/))
*   **Solveurs ADF** (pour `arg.adf.sat.solver.*` - Section 5.1) :
    *   PicoSAT, Lingeling, MiniSat (fournis avec Tweety pour certaines plateformes, voir `libs/`)

**Instructions :**
*   Modifiez les variables `*_PATH` dans la cellule suivante avec les chemins corrects sur **votre** système.
*   Si un outil n'est pas installé ou si vous ne souhaitez pas l'utiliser, laissez le chemin vide (`""`) ou commentez la ligne correspondante. Le notebook essaiera d'utiliser des alternatives internes (plus lentes) ou sautera les sections concernées.
*   Sous Windows, n'oubliez pas l'extension `.exe` et utilisez des anti-slashs (`\\`) ou des slashs (`/`) pour les chemins.

In [4]:
# --- Configuration de base pour les outils externes ---
import os
import pathlib
import platform
import shutil
import subprocess
import urllib.request
import zipfile
import stat

# Repertoire pour les outils externes
EXT_TOOLS_DIR = pathlib.Path("ext_tools")
EXT_TOOLS_DIR.mkdir(exist_ok=True)

# Repertoire pour les bibliotheques natives SAT (pour ADF)
NATIVE_LIBS_DIR = pathlib.Path("../libs/native")
if not NATIVE_LIBS_DIR.exists():
    for alt in [pathlib.Path("libs/native"), pathlib.Path("../../libs/native")]:
        if alt.exists():
            NATIVE_LIBS_DIR = alt
            break

# Dictionnaire pour stocker les chemins des outils externes
EXTERNAL_TOOLS = {
    "SAT_SOLVER": "",
    "SAT_SOLVER_PYTHON": "",  # Nouveau: wrapper Python avec CaDiCaL
    "EPROVER": "",
    "MARCO": "",
    "OPEN_WBO": "",
    "CLINGO": "",
    "SPASS": ""
}

def get_tool_path(tool_name):
    """Retourne le chemin valide d'un outil ou None."""
    path_str = EXTERNAL_TOOLS.get(tool_name, "")
    if not path_str: 
        return None
    if shutil.which(path_str):
        return path_str
    path_obj = pathlib.Path(path_str)
    if path_obj.is_file() and os.access(path_obj, os.R_OK):
        return str(path_obj.resolve())
    return None

system = platform.system()
print(f"Configuration des outils externes pour {system}")
print(f"Repertoire ext_tools: {EXT_TOOLS_DIR.resolve()}")
print(f"Repertoire libs/native: {NATIVE_LIBS_DIR.resolve() if NATIVE_LIBS_DIR.exists() else 'Non trouve'}")

Configuration des outils externes pour Windows
Repertoire ext_tools: D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\Tweety\ext_tools
Repertoire libs/native: D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\libs\native


#### 1.5.1 Clingo - Solveur ASP (Answer Set Programming)

**Clingo** est le solveur ASP de reference du projet [Potassco](https://potassco.org/). 
Il combine le grounder *gringo* et le solveur *clasp*.

**Utilisation dans Tweety** :
- `lp.asp.reasoner.ClingoSolver` : Raisonnement ASP
- Support des programmes avec defauts et contraintes d'integrite

**Installation** : Si non present, telechargement automatique depuis GitHub.

In [5]:
# === Configuration de Clingo ===
print("=== 1. Configuration de Clingo ===")

clingo_dir = EXT_TOOLS_DIR / "clingo"
clingo_dir.mkdir(exist_ok=True)
clingo_exe = clingo_dir / ("clingo.exe" if system == "Windows" else "clingo")

# Verifier si clingo est deja present
clingo_in_path = shutil.which("clingo") or shutil.which("clingo.exe")
if clingo_in_path:
    EXTERNAL_TOOLS["CLINGO"] = clingo_in_path
    print(f"  OK Clingo trouve dans PATH: {clingo_in_path}")
elif clingo_exe.exists():
    EXTERNAL_TOOLS["CLINGO"] = str(clingo_exe.resolve())
    print(f"  OK Clingo deja present: {clingo_exe}")
else:
    # Telecharger automatiquement
    print("  Telechargement de Clingo...")
    clingo_version = "5.4.0"
    
    if system == "Windows":
        clingo_url = f"https://github.com/potassco/clingo/releases/download/v{clingo_version}/clingo-{clingo_version}-win64.zip"
        clingo_archive = clingo_dir / "clingo.zip"
        try:
            urllib.request.urlretrieve(clingo_url, clingo_archive)
            with zipfile.ZipFile(clingo_archive, 'r') as zip_ref:
                zip_ref.extractall(clingo_dir)
            for exe in clingo_dir.rglob("clingo.exe"):
                shutil.move(str(exe), str(clingo_exe))
                EXTERNAL_TOOLS["CLINGO"] = str(clingo_exe.resolve())
                print(f"  OK Clingo installe: {clingo_exe}")
                break
            clingo_archive.unlink(missing_ok=True)
            for d in clingo_dir.glob("clingo-*"):
                if d.is_dir(): shutil.rmtree(d, ignore_errors=True)
        except Exception as e:
            print(f"  Erreur telechargement Clingo: {e}")
    elif system == "Linux":
        import tarfile
        clingo_url = f"https://github.com/potassco/clingo/releases/download/v{clingo_version}/clingo-{clingo_version}-linux-x86_64.tar.gz"
        clingo_archive = clingo_dir / "clingo.tar.gz"
        try:
            urllib.request.urlretrieve(clingo_url, clingo_archive)
            with tarfile.open(clingo_archive, "r:gz") as tar:
                tar.extractall(path=clingo_dir)
            for exe in clingo_dir.rglob("clingo"):
                if exe.is_file():
                    shutil.move(str(exe), str(clingo_exe))
                    os.chmod(clingo_exe, stat.S_IRWXU | stat.S_IRGRP | stat.S_IXGRP)
                    EXTERNAL_TOOLS["CLINGO"] = str(clingo_exe.resolve())
                    print(f"  OK Clingo installe: {clingo_exe}")
                    break
            clingo_archive.unlink(missing_ok=True)
        except Exception as e:
            print(f"  Erreur installation Clingo: {e}")

=== 1. Configuration de Clingo ===
  OK Clingo deja present: ext_tools\clingo\clingo.exe


#### 1.5.2 SPASS - Prouveur de Logique Modale

**SPASS** est un prouveur automatique de theoremes pour la logique du premier ordre 
avec support pour la logique modale.

**Utilisation dans Tweety** :
- `logics.ml.reasoner.SPASSMlReasoner` : Raisonnement en logique modale (Section 2.4)

**Note** : Sans SPASS, `SimpleMlReasoner` peut bloquer indefiniment. 
SPASS est donc fortement recommande pour les notebooks sur la logique modale.

In [6]:
# === Configuration de SPASS ===
print("=== 2. Configuration de SPASS ===")

spass_dir = EXT_TOOLS_DIR / "spass"
spass_dir.mkdir(exist_ok=True)
spass_exe = spass_dir / ("SPASS.exe" if system == "Windows" else "SPASS")

# IMPORTANT: Le fichier spass30windows.exe sur le site SPASS est un INSTALLEUR,
# pas un executable standalone. Il lance une interface d'installation a chaque appel.
# => On ne telecharge PAS automatiquement sous Windows.

spass_in_path = shutil.which("SPASS") or shutil.which("SPASS.exe")
if spass_in_path:
    EXTERNAL_TOOLS["SPASS"] = spass_in_path
    print(f"  OK SPASS trouve dans PATH: {spass_in_path}")
elif spass_exe.exists():
    # Verifier que ce n'est pas l'installeur (taille > 1MB = probablement standalone)
    file_size = spass_exe.stat().st_size
    if file_size > 1_000_000:  # > 1MB = probablement l'installeur
        print(f"  WARN Le fichier {spass_exe} semble etre l'installeur SPASS.")
        print(f"       Taille: {file_size/1024:.0f} KB (attendu ~200-500 KB pour standalone)")
        print(f"       Supprimez-le et installez SPASS manuellement.")
    else:
        EXTERNAL_TOOLS["SPASS"] = str(spass_exe.resolve())
        print(f"  OK SPASS deja present: {spass_exe}")
else:
    if system == "Windows":
        # NE PAS telecharger automatiquement - le fichier est un installeur, pas un executable
        print("  INFO SPASS non configure.")
        print("  Pour Windows, installez SPASS manuellement:")
        print("    1. Telecharger depuis: https://www.spass-prover.org/download/binaries/")
        print("    2. Choisir 'spass30windows.exe' et EXECUTER l'installeur")
        print("    3. Copier SPASS.exe du repertoire d'installation vers:")
        print(f"       {spass_dir}")
        print("    4. Re-executer cette cellule")
        print("")
        print("  Note: SPASS est OPTIONNEL. La logique modale fonctionne sans raisonnement.")
    elif system == "Linux":
        import tarfile
        arch = "64" if platform.architecture()[0] == "64bit" else "32"
        spass_url = f"https://www.spass-prover.org/download/binaries/spass35pclinux{arch}.tgz"
        spass_archive = spass_dir / "spass.tgz"
        print(f"  Telechargement de SPASS pour Linux ({arch}bit)...")
        try:
            urllib.request.urlretrieve(spass_url, spass_archive)
            with tarfile.open(spass_archive, "r:gz") as tar:
                tar.extractall(path=spass_dir)
            extracted = spass_dir / "SPASS" / "SPASS"
            if extracted.exists():
                shutil.move(str(extracted), str(spass_exe))
                os.chmod(spass_exe, stat.S_IRWXU | stat.S_IRGRP | stat.S_IXGRP)
                EXTERNAL_TOOLS["SPASS"] = str(spass_exe.resolve())
                print(f"  OK SPASS installe: {spass_exe}")
            spass_archive.unlink(missing_ok=True)
            shutil.rmtree(spass_dir / "SPASS", ignore_errors=True)
        except Exception as e:
            print(f"  Erreur installation SPASS: {e}")
    elif system == "Darwin":
        print("  INFO SPASS non disponible en telechargement automatique pour macOS.")
        print("  Installez manuellement depuis: https://www.spass-prover.org/")

=== 2. Configuration de SPASS ===
  OK SPASS deja present: ext_tools\spass\SPASS.exe


#### 1.5.3 EProver - Prouveur FOL de Haute Performance

**E Prover** est un prouveur automatique de theoremes pour la logique du premier ordre (FOL). 
C'est l'un des prouveurs les plus performants, regulierement gagnant des competitions CASC.

**Utilisation dans Tweety** :
- `logics.fol.reasoner.EFOLReasoner` : Raisonnement FOL avance
- Alternative a `SimpleFolReasoner` qui peut avoir des problemes de memoire

**Note** : EProver est fourni dans le depot (ext_tools/EProver/).

In [7]:
# === Configuration de EProver (prouveur FOL) ===
print("=== 3. Configuration de EProver ===")

eprover_search_paths = [
    pathlib.Path("../ext_tools/EProver"),
    pathlib.Path("ext_tools/EProver"),
    pathlib.Path("../../ext_tools/EProver"),
]
eprover_exe_name = "eprover.exe" if system == "Windows" else "eprover"

eprover_in_path = shutil.which("eprover") or shutil.which("eprover.exe")
if eprover_in_path:
    EXTERNAL_TOOLS["EPROVER"] = eprover_in_path
    print(f"  OK EProver trouve dans PATH: {eprover_in_path}")
else:
    for search_path in eprover_search_paths:
        eprover_exe = search_path / eprover_exe_name
        if eprover_exe.exists():
            EXTERNAL_TOOLS["EPROVER"] = str(eprover_exe.resolve())
            print(f"  OK EProver trouve: {eprover_exe.resolve()}")
            break
    else:
        print("  EProver non trouve dans les chemins standards.")
        print("  Telecharger depuis https://eprover.org/")

=== 3. Configuration de EProver ===
  OK EProver trouve: D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\ext_tools\EProver\eprover.exe


#### 1.5.4 Solveurs SAT - Satisfaction Booleenne

Les **solveurs SAT** sont au coeur de nombreuses fonctionnalites de Tweety :
- Raisonnement en logique propositionnelle
- Calcul des extensions d'argumentation (ADF)
- Mesures d'incoherence

**Hierarchie des solveurs** (du plus rapide au plus portable) :

| Solveur | Type | Performance | Portabilite |
|---------|------|-------------|-------------|
| CaDiCaL 1.9.5 | Python (pySAT) | Champion SAT | Multi-plateforme |
| Lingeling | DLL native | Tres rapide | Windows/Linux |
| MiniSat | DLL native | Rapide | Windows/Linux |
| PicoSAT | DLL native | Rapide | Windows/Linux |
| Sat4j | Pure Java | Modere | Multi-plateforme |

**Nouveau** : Le wrapper `sat_solver.py` donne acces aux solveurs modernes via pySAT.

In [8]:
# === Configuration des solveurs SAT ===
print("=== 4. Configuration des solveurs SAT ===")

# 4.1 Bibliotheques SAT natives (pour ADF via JNI)
print("\n--- 4.1 Bibliotheques SAT natives ---")
if NATIVE_LIBS_DIR.exists():
    print(f"  Repertoire libs/native: {NATIVE_LIBS_DIR.resolve()}")
    sat_libs_found = []
    for solver in ["lingeling", "minisat", "picosat"]:
        dll = NATIVE_LIBS_DIR / f"{solver}.dll"
        so = NATIVE_LIBS_DIR / f"{solver}.so"
        if dll.exists(): sat_libs_found.append(f"{solver}.dll")
        elif so.exists(): sat_libs_found.append(f"{solver}.so")
    if sat_libs_found:
        print(f"  OK Bibliotheques trouvees: {', '.join(sat_libs_found)}")
    else:
        print("  Aucune bibliotheque native trouvee")
else:
    print("  Repertoire libs/native non trouve")

# 4.2 Sat4j (Pure Java - toujours disponible)
print("\n--- 4.2 Sat4j (integre) ---")
EXTERNAL_TOOLS["SAT_SOLVER"] = "Sat4j (integre)"
print("  OK Sat4j disponible (org.tweetyproject.logics.pl.sat.Sat4jSolver)")

# 4.3 Wrapper Python avec CaDiCaL et autres (nouveau!)
print("\n--- 4.3 Wrapper Python SAT (CaDiCaL, Glucose, etc.) ---")
sat_solver_paths = [
    pathlib.Path("../ext_tools/sat_solver.py"),
    pathlib.Path("ext_tools/sat_solver.py"),
]
for sp in sat_solver_paths:
    if sp.exists():
        EXTERNAL_TOOLS["SAT_SOLVER_PYTHON"] = str(sp.resolve())
        print(f"  OK sat_solver.py trouve: {sp.resolve()}")
        try:
            from pysat.solvers import SolverNames
            solvers = ['cadical195', 'glucose42', 'maplechrono', 'lingeling']
            available = [s for s in solvers if hasattr(SolverNames, s)]
            print(f"  OK Solveurs disponibles: {', '.join(available)}")
        except ImportError:
            print("  WARN python-sat non disponible - redemarrez le noyau apres section 1.3")
        break
else:
    print("  sat_solver.py non trouve")

=== 4. Configuration des solveurs SAT ===

--- 4.1 Bibliotheques SAT natives ---
  Repertoire libs/native: D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\libs\native
  OK Bibliotheques trouvees: lingeling.dll, minisat.dll, picosat.dll

--- 4.2 Sat4j (integre) ---
  OK Sat4j disponible (org.tweetyproject.logics.pl.sat.Sat4jSolver)

--- 4.3 Wrapper Python SAT (CaDiCaL, Glucose, etc.) ---
  OK sat_solver.py trouve: D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\ext_tools\sat_solver.py
  OK Solveurs disponibles: cadical195, glucose42, maplechrono, lingeling


#### 1.5.5 MARCO et MaxSAT - Analyse d'Incoherence

Ces outils sont utilises pour l'analyse des bases de croyances incoherentes :

**MARCO** (MUS enumerator) :
- Enumere les **MUS** (Minimal Unsatisfiable Subsets) et **MCS** (Maximal Consistent Subsets)
- Utilise Z3 comme backend
- Utilise par `MarcoMusEnumerator` dans Tweety

**MaxSAT** :
- Resout le probleme d'optimisation : maximiser le nombre de clauses satisfaites
- Wrapper Python utilisant l'algorithme RC2 (gagnant MaxSAT Evaluation 2018)
- Utilise par les mesures d'incoherence basees sur MinSum

In [9]:
# === Configuration de MARCO et MaxSAT ===
print("=== 5. Configuration de MARCO (MUS enumerator) ===")

marco_search_paths = [
    pathlib.Path("../ext_tools/marco.py"),
    pathlib.Path("ext_tools/marco.py"),
]
for mp in marco_search_paths:
    if mp.exists():
        EXTERNAL_TOOLS["MARCO"] = str(mp.resolve())
        print(f"  OK MARCO trouve: {mp.resolve()}")
        try:
            import z3
            print(f"  OK Z3 disponible (version {z3.get_version_string()})")
        except ImportError:
            print("  WARN z3 non disponible - redemarrez le noyau apres section 1.3")
        break
else:
    print("  MARCO non trouve")

print("\n=== 6. Configuration de MaxSAT ===")
maxsat_search_paths = [
    pathlib.Path("../ext_tools/maxsat_solver.py"),
    pathlib.Path("ext_tools/maxsat_solver.py"),
]
for mp in maxsat_search_paths:
    if mp.exists():
        EXTERNAL_TOOLS["OPEN_WBO"] = str(mp.resolve())
        print(f"  OK MaxSAT solver trouve: {mp.resolve()}")
        try:
            import pysat
            print(f"  OK python-sat disponible (RC2 algorithm)")
        except ImportError:
            print("  WARN python-sat non disponible - redemarrez le noyau apres section 1.3")
        break
else:
    print("  MaxSAT solver non trouve")

# === Resume final ===
print("\n" + "="*50)
print("RESUME DES OUTILS EXTERNES")
print("="*50)
for tool, path in EXTERNAL_TOOLS.items():
    status = "OK" if (path and (path.startswith("Sat4j") or get_tool_path(tool))) else "Non configure"
    display_path = path[:50] + "..." if path and len(path) > 50 else (path or "-")
    print(f"  {tool:<18}: [{status:^12}] {display_path}")
print("="*50)

=== 5. Configuration de MARCO (MUS enumerator) ===
  OK MARCO trouve: D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\ext_tools\marco.py
  OK Z3 disponible (version 4.15.4)

=== 6. Configuration de MaxSAT ===
  OK MaxSAT solver trouve: D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\ext_tools\maxsat_solver.py
  OK python-sat disponible (RC2 algorithm)

RESUME DES OUTILS EXTERNES
  SAT_SOLVER        : [     OK     ] Sat4j (integre)
  SAT_SOLVER_PYTHON : [     OK     ] D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\ext_to...
  EPROVER           : [     OK     ] D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\ext_to...
  MARCO             : [     OK     ] D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\ext_to...
  OPEN_WBO          : [     OK     ] D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\ext_to...
  CLINGO            : [     OK     ] D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\Tweety...
  SPASS             : [     OK     ] D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\Tweety...


### 1.6 Démarrage de la JVM via JPype
<a id="1.6"></a>

JPype permet à Python d'interagir directement avec le code Java. Nous devons démarrer une Machine Virtuelle Java (JVM) et lui indiquer où trouver les fichiers JAR de Tweety que nous venons de télécharger.

* Le code tente de trouver votre `JAVA_HOME`. Modifiez `java_home_path` si nécessaire dans la cellule suivante.
* Il construit le `classpath` Java en listant tous les `.jar` dans le dossier `libs/`.
* `jpype.startJVM()` lance la JVM. Si elle est déjà lancée (ex: après redémarrage du noyau), l'appel est ignoré.
* **Crucial**: `jpype.imports.registerDomain(...)` doit être appelé **après** le démarrage de la JVM pour permettre les imports courts (ex: `from org.tweetyproject...`).

#### 1.6.1 Fonctions Utilitaires JDK

Avant de demarrer la JVM, nous definissons des fonctions pour localiser ou telecharger automatiquement un JDK compatible.

**Strategie de detection JDK** (par ordre de priorite) :
1. JDK portable existant dans `jdk-17-portable/`
2. Telechargement automatique de Zulu JDK 17 (Azul)
3. Variable d'environnement `JAVA_HOME`
4. Detection automatique des JDKs systeme

In [10]:
# --- 1.6.1 Fonctions Utilitaires JDK ---
import jpype
import jpype.imports
import os
import pathlib
import platform
import urllib.request
import zipfile
import shutil
from jpype.types import *
import stat

# URLs de telechargement Zulu JDK 17 (Azul)
JDK_DOWNLOAD_URLS = {
    "Windows": "https://cdn.azul.com/zulu/bin/zulu17.50.19-ca-jdk17.0.11-win_x64.zip",
    "Linux": "https://cdn.azul.com/zulu/bin/zulu17.50.19-ca-jdk17.0.11-linux_x64.tar.gz",
    "Darwin": "https://cdn.azul.com/zulu/bin/zulu17.50.19-ca-jdk17.0.11-macosx_x64.zip"
}

def find_portable_jdk():
    """Localise le JDK portable dans l'arborescence du projet."""
    print("Recherche JDK portable dans l'arborescence projet...")
    search_paths = [
        pathlib.Path("jdk-17-portable"),
        pathlib.Path("../jdk-17-portable"),
        pathlib.Path("../Argument_Analysis/jdk-17-portable"),
        pathlib.Path("../../jdk-17-portable"),
    ]
    exe_suffix = ".exe" if platform.system() == "Windows" else ""
    for base_path in search_paths:
        if base_path.exists():
            for pattern in ["*jdk*", "zulu*"]:
                for jdk_dir in base_path.glob(pattern):
                    if jdk_dir.is_dir():
                        java_exe = jdk_dir / "bin" / f"java{exe_suffix}"
                        if java_exe.exists():
                            print(f"  OK JDK portable trouve: {jdk_dir.absolute()}")
                            return str(jdk_dir.absolute())
    print("  JDK portable non trouve")
    return None

def download_portable_jdk():
    """Telecharge et extrait le JDK portable Zulu 17."""
    system = platform.system()
    if system not in JDK_DOWNLOAD_URLS:
        print(f"Systeme {system} non supporte pour telechargement auto")
        return None
    url = JDK_DOWNLOAD_URLS[system]
    jdk_dir = pathlib.Path("jdk-17-portable")
    jdk_dir.mkdir(exist_ok=True)
    archive_name = url.split("/")[-1]
    archive_path = jdk_dir / archive_name
    print(f"Telechargement JDK depuis {url}...")
    try:
        urllib.request.urlretrieve(url, archive_path)
        print(f"Extraction de {archive_name}...")
        if archive_name.endswith(".zip"):
            with zipfile.ZipFile(archive_path, 'r') as zf:
                zf.extractall(jdk_dir)
        elif archive_name.endswith(".tar.gz"):
            import tarfile
            with tarfile.open(archive_path, 'r:gz') as tf:
                tf.extractall(jdk_dir)
        archive_path.unlink()
        return find_portable_jdk()
    except Exception as e:
        print(f"Erreur telechargement JDK: {e}")
        return None

print("Fonctions JDK definies: find_portable_jdk(), download_portable_jdk()")

Fonctions JDK definies: find_portable_jdk(), download_portable_jdk()


#### 1.6.2 Detection et Configuration du JDK

Cette cellule detecte le JDK disponible et le telecharge si necessaire.

**Important** : JPype necessite un JDK 11+ (JDK 17 recommande pour Tweety 1.29).

In [11]:
# --- 1.6.2 Detection du JDK ---

def find_java_home():
    """Trouve JAVA_HOME selon la strategie de priorite."""
    # 1. JDK portable existant
    portable_jdk = find_portable_jdk()
    if portable_jdk:
        os.environ['JAVA_HOME'] = portable_jdk
        return portable_jdk
    
    # 2. Telechargement automatique
    print("JDK portable non trouve - telechargement automatique...")
    downloaded_jdk = download_portable_jdk()
    if downloaded_jdk:
        os.environ['JAVA_HOME'] = downloaded_jdk
        return downloaded_jdk
    
    # 3. Variable JAVA_HOME
    java_home_env = os.getenv("JAVA_HOME")
    exe_suffix = ".exe" if platform.system() == "Windows" else ""
    if java_home_env and pathlib.Path(java_home_env).is_dir():
        if (pathlib.Path(java_home_env) / "bin" / f"java{exe_suffix}").exists():
            print(f"  Utilisation JAVA_HOME: {java_home_env}")
            return java_home_env
    
    # 4. Detection systeme
    print("Detection automatique des JDKs systeme...")
    possible_locations = []
    if platform.system() == "Windows":
        java_dir = pathlib.Path("C:/Program Files/Java/")
        if java_dir.is_dir():
            possible_locations = sorted(java_dir.glob("jdk-*/"), reverse=True)
    elif platform.system() == "Linux":
        for p_str in ["/usr/lib/jvm"]:
            p_obj = pathlib.Path(p_str)
            if p_obj.is_dir():
                possible_locations.extend(sorted(p_obj.glob("java-*"), reverse=True))
    for p in possible_locations:
        java_bin = p / "bin" / f"java{exe_suffix}"
        if java_bin.exists():
            print(f"  JDK systeme detecte: {p}")
            os.environ['JAVA_HOME'] = str(p)
            return str(p)
    
    print("ERREUR: Aucun JDK trouve ou telechargeable")
    return None

# Executer la detection
java_home_path = find_java_home()
if java_home_path:
    print(f"\nJAVA_HOME configure: {java_home_path}")

Recherche JDK portable dans l'arborescence projet...
  OK JDK portable trouve: d:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\Tweety\jdk-17-portable\zulu17.50.19-ca-jdk17.0.11-win_x64

JAVA_HOME configure: d:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\Tweety\jdk-17-portable\zulu17.50.19-ca-jdk17.0.11-win_x64


#### 1.6.3 Construction du Classpath

Le **classpath** indique a la JVM ou trouver les classes Java.
Nous assemblons tous les JARs Tweety du dossier `libs/`.

In [12]:
# --- 1.6.3 Construction du Classpath ---
classpath_separator = os.pathsep
if 'LIB_DIR' not in globals():
    LIB_DIR = pathlib.Path("libs")

jar_list = [str(p.resolve()) for p in LIB_DIR.glob("*.jar")]
num_jars_found = len(jar_list)

if not jar_list:
    print("ERREUR: Aucun JAR trouve dans libs/")
    classpath = ""
else:
    classpath = classpath_separator.join(jar_list)
    print(f"Classpath assemble: {num_jars_found} JARs")
    
    # Verification presence JAR beliefdynamics
    beliefdynamics_found = any("beliefdynamics" in j for j in jar_list)
    if beliefdynamics_found:
        print("  OK JAR beliefdynamics present")
    else:
        print("  ATTENTION: JAR beliefdynamics non trouve")

Classpath assemble: 35 JARs
  OK JAR beliefdynamics present


#### 1.6.4 Demarrage de la JVM

**Important** : Si des JARs ont ete telecharges dans les cellules precedentes,
**redemarrez le noyau** avant d'executer cette cellule.

La JVM est demarree une seule fois par session Python. Une fois demarree,
toutes les classes Java sont accessibles via les imports Python.

In [13]:
# --- 1.6.4 Demarrage de la JVM ---

if not jpype.isJVMStarted():
    if not java_home_path:
        print("ERREUR: Impossible de demarrer la JVM sans JAVA_HOME")
    elif num_jars_found == 0:
        print("ERREUR: Classpath vide - verifiez le dossier libs/")
    else:
        print("Demarrage de la JVM...")
        jvm_args = ["-ea", f"-Djava.class.path={classpath}"]
        
        # Ajouter bibliotheques natives si presentes
        if 'NATIVE_LIBS_DIR' in globals() and NATIVE_LIBS_DIR.exists():
            jvm_args.append(f"-Djava.library.path={NATIVE_LIBS_DIR.resolve()}")
            print(f"  Avec bibliotheques natives: {NATIVE_LIBS_DIR}")
        
        try:
            jpype.startJVM(*jvm_args, convertStrings=False)
            jpype.imports.registerDomain("org")
            jpype.imports.registerDomain("java")
            jpype.imports.registerDomain("net")
            print("OK JVM demarree et domaines enregistres")
        except Exception as e:
            print(f"ERREUR demarrage JVM: {e}")
else:
    print("JVM deja en cours d'execution")

Demarrage de la JVM...
  Avec bibliotheques natives: ..\libs\native
OK JVM demarree et domaines enregistres


#### 1.6.5 Verification des Imports Java

Verification que les classes Tweety essentielles sont accessibles.

**Note sur InformationObject** : Dans Tweety 1.29, cette classe se trouve dans
`org.tweetyproject.beliefdynamics.mas.InformationObject` (package `mas`).
CrMas et la Revision de Croyances multi-agents fonctionnent normalement.

In [14]:
# --- 1.6.5 Verification des Imports ---
print("Test des imports Java essentiels...")

CrMas_Imports_OK = False  # Variable utilisee par les autres notebooks

if jpype.isJVMStarted():
    imports_ok = True
    missing = []
    
    # Test InformationObject (dans le package 'mas' depuis Tweety 1.28+)
    try:
        from org.tweetyproject.beliefdynamics.mas import InformationObject, CrMasBeliefSet, CrMasRevisionWrapper
        print("  OK InformationObject (org.tweetyproject.beliefdynamics.mas)")
        print("  OK CrMasBeliefSet")
        print("  OK CrMasRevisionWrapper")
        CrMas_Imports_OK = True
    except ImportError as e:
        print(f"  INFO CrMas imports incomplets: {e}")
        print("       Impact: Seule la section CrMas sera desactivee.")
    
    # Tests critiques (doivent tous reussir)
    try:
        from org.tweetyproject.commons import Formula
        print("  OK commons.Formula")
    except ImportError:
        missing.append("commons.Formula")
        imports_ok = False
    
    try:
        from org.tweetyproject.logics.pl.syntax import Proposition, PlFormula
        print("  OK logics.pl.syntax (Proposition, PlFormula)")
    except ImportError:
        missing.append("logics.pl.syntax")
        imports_ok = False
    
    try:
        from org.tweetyproject.arg.dung.syntax import Argument, DungTheory
        print("  OK arg.dung.syntax (Argument, DungTheory)")
    except ImportError:
        missing.append("arg.dung.syntax")
        imports_ok = False
    
    try:
        from java.util import ArrayList, HashSet
        print("  OK java.util (ArrayList, HashSet)")
    except ImportError:
        missing.append("java.util")
        imports_ok = False
    
    # Resume
    print("")
    if imports_ok:
        print("Tous les imports critiques sont disponibles.")
        print("Tweety est pret a etre utilise!")
    else:
        print(f"ERREUR: Imports manquants: {', '.join(missing)}")
        print("Verifiez les JARs dans libs/")
    
    # Afficher le statut CrMas
    tweety_ver = globals().get('TWEETY_VERSION', 'inconnue')
    print(f"\nVersion Tweety: {tweety_ver}")
    print(f"CrMas_Imports_OK = {CrMas_Imports_OK}")
    if not CrMas_Imports_OK:
        print("(Les sections CrMas seront sautees automatiquement)")
else:
    print("JVM non demarree - impossible de tester les imports")

Test des imports Java essentiels...
  OK InformationObject (org.tweetyproject.beliefdynamics.mas)
  OK CrMasBeliefSet
  OK CrMasRevisionWrapper
  OK commons.Formula
  OK logics.pl.syntax (Proposition, PlFormula)
  OK arg.dung.syntax (Argument, DungTheory)
  OK java.util (ArrayList, HashSet)

Tous les imports critiques sont disponibles.
Tweety est pret a etre utilise!

Version Tweety: 1.29
CrMas_Imports_OK = True


### 1.7 Concepts Clés de Tweety et JPype
<a id="1.7"></a>

Avant de plonger dans les exemples, comprenons quelques concepts fondamentaux de Tweety et comment nous interagissons avec eux via JPype :

**Concepts Tweety :**

*   **Signature** (`org.tweetyproject.logics.[logic].syntax.[Logic]Signature`) : Définit le vocabulaire d'une logique.
*   **Formula** (`org.tweetyproject.logics.[logic].syntax.[Logic]Formula`) : Représente une formule bien formée.
*   **BeliefBase** (`org.tweetyproject.logics.[logic].syntax.[Logic]BeliefSet`) : Un ensemble de formules (base de connaissances).
*   **Interpretation** (`org.tweetyproject.logics.[logic].semantics.Interpretation`) : Assigne une signification sémantique (ex: `PossibleWorld`).
*   **Parser** (`org.tweetyproject.logics.[logic].parser.[Logic]Parser`) : Analyse chaînes/fichiers vers `Formula`/`BeliefSet`.
*   **Reasoner** (`org.tweetyproject.logics.[logic].reasoner.[Logic]Reasoner`) : Implémente le raisonnement (`query`, `getModels`). Des raisonneurs par défaut peuvent être définis.
*   **Argumentation Frameworks** (`org.tweetyproject.arg.*`): Classes pour les cadres (`DungTheory`, `AspicArgumentationTheory`, etc.) et composants (`Argument`, `Attack`, `Support`).

**Interaction Python-Java avec JPype :**

*   **Imports**: Grâce à `jpype.imports.registerDomain("org", alias="org")` (exécuté dans la cellule de démarrage JVM), on peut importer les classes Java comme des modules Python :
    ```python
    from org.tweetyproject.logics.pl.syntax import Proposition
    from org.tweetyproject.arg.dung.syntax import Argument, Attack
    from java.util import ArrayList # Également possible via registerDomain("java", alias="java")
    ```
*   **Instanciation**: `p = Proposition("a")`.
*   **Appel de Méthodes**: `kb.add(p)`.
*   **Types Primitifs**: Conversion auto (int, float, bool, str).
*   **Collections Java**: Utiliser les types Java explicites (`ArrayList`, `HashSet` depuis `java.util`).
*   **Surcharge (Overloading)**: Si ambiguïté, caster avec `JObject(variable, ClasseJava)` ou `JInt()`, `JString()`, etc. (depuis `jpype.types`).
*   **Exceptions Java**: Attraper avec `except jpype.JException as e_java:`. Accéder au message avec `e_java.message()` et à la trace avec `e_java.stacktrace()`.

---

## Notebooks de la série Tweety

Après avoir exécuté ce notebook de configuration, vous pouvez explorer:

| # | Notebook | Thème |
|---|----------|-------|
| 2 | [Tweety-2-Basic-Logics](Tweety-2-Basic-Logics.ipynb) | Logique Propositionnelle et FOL |
| 3 | [Tweety-3-Advanced-Logics](Tweety-3-Advanced-Logics.ipynb) | DL, Modale, QBF, Conditional |
| 4 | [Tweety-4-Belief-Revision](Tweety-4-Belief-Revision.ipynb) | Révision de croyances, MUS, MaxSAT |
| 5 | [Tweety-5-Abstract-Argumentation](Tweety-5-Abstract-Argumentation.ipynb) | Dung, CF2, Génération |
| 6 | [Tweety-6-Structured-Argumentation](Tweety-6-Structured-Argumentation.ipynb) | ASPIC+, DeLP, ABA, ASP |
| 7a | [Tweety-7a-Extended-Frameworks](Tweety-7a-Extended-Frameworks.ipynb) | ADF, Bipolar, WAF, SAF, SetAF, Extended |
| 7b | [Tweety-7b-Ranking-Probabilistic](Tweety-7b-Ranking-Probabilistic.ipynb) | Ranking Semantics, Probabiliste |
| 8 | [Tweety-8-Agent-Dialogues](Tweety-8-Agent-Dialogues.ipynb) | Agents, Dialogues argumentatifs |
| 9 | [Tweety-9-Preferences](Tweety-9-Preferences.ipynb) | Préférences, Théorie du vote |

---

**Navigation**: Index | [Suivant: Tweety-2-Basic-Logics →](Tweety-2-Basic-Logics.ipynb)